# Propuesta de trabajo

## Participantes

* Cristobal Cancino
* Tomás Cortés
* Bruno Martinez

### Presentación de los datos, definición de variables y contexto de la problemática.

Se utilizará una base de datos sobre proteinas entregada por _Research Collaboratory for Structural Bioinformatics (RCSB)_ de _Protein Data Bank (PDB)_, la cual describe proteinas y otras macromoléculas biológicas importantes. Para determinar el lugar de cada átomo en relación a otros en la molécula, biólogos utilizan métodos tales como Cristalografía X-Ray, Espectroscopia NMR y microoscopia crio-electrón. 

Esta base de datos sobre proteinas ha ayudado a la comunidad cientifica para estudiar diferentes enfermedades y preparar nuevos medicamentos y soluciones que han ayudado.

El archivo PDB es un depósito de coordenadas atómicas y otra información que describe proteínas y otras macromoléculas biológicas importantes. Los biólogos estructurales utilizan métodos como la cristalografía de rayos X, la espectroscopia de RMN y la microscopía crioelectrónica para determinar la ubicación de cada átomo en relación con los demás en la molécula. Luego depositan esta información, que luego es anotada y publicada públicamente en el archivo por wwPDB.

El PDB en constante crecimiento es un reflejo de la investigación que se lleva a cabo en los laboratorios de todo el mundo. Esto puede hacer que el uso de la base de datos en la investigación y la educación sea emocionante y desafiante. Las estructuras están disponibles para muchas de las proteínas y ácidos nucleicos involucrados en los procesos centrales de la vida, por lo que puede ir al archivo de PDB para encontrar estructuras para ribosomas, oncogenes, dianas de fármacos e incluso virus completos. Sin embargo, puede ser un desafío encontrar la información que necesita, ya que la PDB archiva tantas estructuras diferentes. A menudo encontrará múltiples estructuras para una molécula determinada, o estructuras parciales, o estructuras que han sido modificadas o inactivadas de su forma nativa.

Recuperado de [Structural Protein Sequence](https://www.kaggle.com/shahir/protein-data-set?select=pdb_data_no_dups.csv)

In [13]:
import os

import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt

In [52]:
data_path = os.path.join("data", "pdb_data_no_dups.csv")
pdb_no_dups = pd.read_csv(data_path, index_col=0).dropna()
#sum(pdb_no_dups['classification'].value_counts()[:10]) / pdb_no_dups['classification'].count()
pdb_no_dups.head()

,classification,experimentalTechnique,macromoleculeType,residueCount,resolution,structureMolecularWeight,crystallizationMethod,crystallizationTempK,densityMatthews,densityPercentSol,pdbxDetails,phValue,publicationYear
structureId,,,,,,,,,,,,,
102D,DNA,X-RAY DIFFRACTION,DNA,24,2.20,7637.17,"VAPOR DIFFUSION, SITTING DROP",277.0,2.28,46.06,"pH 7.00, VAPOR DIFFUSION, SITTING DROP, temper...",7.0,1995.0
110D,DNA,X-RAY DIFFRACTION,DNA,6,1.90,2337.73,"VAPOR DIFFUSION, SITTING DROP",277.0,2.90,57.63,"pH 6.60, VAPOR DIFFUSION, SITTING DROP, temper...",6.6,1993.0
111D,DNA,X-RAY DIFFRACTION,DNA,24,2.25,7374.83,"VAPOR DIFFUSION, SITTING DROP",277.0,2.29,46.25,"pH 6.60, VAPOR DIFFUSION, SITTING DROP, temper...",6.6,1989.0
113D,DNA,X-RAY DIFFRACTION,DNA,24,2.50,7356.81,"VAPOR DIFFUSION, SITTING DROP",281.0,2.35,47.59,"pH 7.40, VAPOR DIFFUSION, SITTING DROP, temper...",7.4,1987.0
117D,DNA,X-RAY DIFFRACTION,DNA,12,2.55,3663.39,VAPOR DIFFUSION,277.0,3.01,59.09,"pH 6.50, VAPOR DIFFUSION, temperature 277.00K",6.5,1992.0


Esta base de datos contiene las siguientes columnas

1. **structureid**: Indexación para identificar la estructura en particular.
2. **clasification**: Estructura en estudio.
3. **experimentalTechnique**: Técnica usada para determinar la ubicación relativa entre cada átomo de la molecula.
4. **macromoleculeType**: Tipo de macromolecula correspondiente a la estructura
5. **residueCount**: Número de residuos, es decir, el número de monómeros que componen la macromolécula.
6. **resolution**: Resolución del microscopío. Para cada resolución, se tiene la siguiente interpretación

| Resolucion | Interpretación                                                                                                                                                                                                              |
|------------|-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| >4.0       | Coordenadas individuales sin significado                                                                                                                                                                                    |
| 3.0 - 4.0  | Plegamiento posiblemente correcto, pero comúnmente con errores. Algunas cadenas laterales poseen mal los rotámeros.                                                                                                         |
| 2.5 - 3.0  | Plegamiento bien dilucidado salvo en algunos pliegues superficiales, mal modelados. Algunas cadenas laterales largas (Lys, Glu, Gln) y otraqs cortas (Ser, Val, Thr) mal orientadas.                                        |
| 2.0 - 2.5  | El número de cadenas laterales con un rotámero incorrecto es mucho menor. Los errores, pequeños, son detectados normalmente. Los pliegues superficiales están bastante bien definidos. Los ligandos y el agua son visibles. |
| 1.5 - 2.0  | Pocos residuos poseen mal rotámero. Los errores pequeños son detectados. Los pliegues incorrectos son muy raros, incluso en superficie.                                                                                     |
| 0.5 - 1.5  | En general, todo está corrctamente resuelto. Las librerías de rotámeros y los estudios geométricos se hacen a este nivel de precisión.                                                                                      |

7. **structureMoleculeWeight**: Masa molar de la estructura
8. **cristalizationMethod**: Método de critalización. Aquellos experimentos que no consideran cristalización les corresponde NaN.
9. **crystalizationTemperature**: Temperatura de cristalización. Idem.
10. **densityMatthews**: Densidad de Matthews, dada por $$\frac{\text{volumen}}{\text{peso molecular} * Z * X},$$ donde $Z$ es el número de unidades asimétricas en la unidad celular y $X$ es el número de moleculas en la unidad asimétrica.
11. **densitityPercentSol**: Porcentaje de densidad de la solución
12. **pdbxDetails**: Detalles extra del experimento
13. **phValue**: Valor del ph
14. **publicationYear**: Año de publicación

In [53]:
data_path = os.path.join("data", "pdb_data_seq.csv")
pdb_seq = pd.read_csv(data_path, index_col=0).dropna()
pdb_seq.head()

,chainId,sequence,residueCount,macromoleculeType
structureId,,,,
100D,A,CCGGCGCCGG,20,DNA/RNA Hybrid
100D,B,CCGGCGCCGG,20,DNA/RNA Hybrid
101D,A,CGCGAATTCGCG,24,DNA
101D,B,CGCGAATTCGCG,24,DNA
101M,A,MVLSEGEWQLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDR...,154,Protein


Esta base de datos contiene las siguientes columnas

1. **structureid**: Indexación para identificar la estructura en particular.
2. **chainId**: Cadena de la estructura.
3. **sequence**: Secuencia molecular de la macromolécula.
4. **macromoleculeType**: Tipo de macromolecula correspondiente a la estructura
5. **residueCount**: Número de residuos, es decir, el número de monómeros que componen la macromolécula.

### ¿Cuál es la pregunta que deseamos resolver?

### Análisis estadístico de los datos.

In [22]:
pdb_no_dups.describe(include="all").T.fillna("")

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
classification,65886.0,2511,HYDROLASE,10237,,,,,,,
experimentalTechnique,65886.0,9,X-RAY DIFFRACTION,65851,,,,,,,
macromoleculeType,65886.0,13,Protein,60877,,,,,,,
residueCount,65886.0,,,,811.165,1636.47,5,279,463,852,89160
resolution,65886.0,,,,2.15498,0.583201,0.48,1.79,2.06,2.5,9.5
structureMolecularWeight,65886.0,,,,100587,288867,488.54,32324.8,53260.7,97861.3,1.24901e+07
crystallizationMethod,65886.0,406,"VAPOR DIFFUSION, HANGING DROP",40076,,,,,,,
crystallizationTempK,65886.0,,,,291.116,9.12086,4,290,293,295,398
densityMatthews,65886.0,,,,2.67706,0.705612,0,2.22,2.5,2.92,12.7
densityPercentSol,65886.0,,,,51.5891,9.97203,0,44.54,50.65,57.91,90.33


In [24]:
pdb_no_dups.dtypes

classification               object
experimentalTechnique        object
macromoleculeType            object
residueCount                  int64
resolution                  float64
structureMolecularWeight    float64
crystallizationMethod        object
crystallizationTempK        float64
densityMatthews             float64
densityPercentSol           float64
pdbxDetails                  object
phValue                     float64
publicationYear             float64
dtype: object

In [23]:
pdb_seq.describe(include="all").T.fillna("")

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
chainId,432474.0,2835,A,132286,,,,,,,
sequence,432474.0,104328,PIVQNLQGQMVHQAISPRTLNAWVKVVEEKAFSPEVIPMFSALSEG...,2580,,,,,,,
residueCount,432474.0,,,,6166.81,24556.4,2,429,1000,3455,313236
macromoleculeType,432474.0,13,Protein,345175,,,,,,,


In [25]:
pdb_seq.dtypes

chainId              object
sequence             object
residueCount          int64
macromoleculeType    object
dtype: object

### Visualización de variables.

### ¿Hay que realizar preprocesamiento?

### ¿Hay que realizar inputación de datos faltantes?

### ¿Hay que realizar balanceo de datos?

### Propuesta de cuatro modelos y sus hiperparámetros asociados.